In [ ]:
# !pip install transformers datasets scikit-learn torch
# !pip install huggingface_hub[hf_xet]

In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset
import torch
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from transformers import TrainingArguments, Trainer, AutoModel, AutoTokenizer

In [ ]:
# Load dataframe
# IMPORTANT: Make sure to put data file in the correct place!
# IMPORTANT: Modify this notebook to loop through all datasets and implement cross-validation!
# IMPORTANT: Should we look into hyperparameter tuning?
df = pd.read_csv("Data/prepended_v3_lemmatized.csv")
df.drop(columns=['text', 'text_length', 'length'], inplace=True)
df.rename(columns={'processed_text':'text', 'target':'label'}, inplace=True)
df = df[["text", "label"]].copy()  # Drop other columns if they exist
df["text"] = df["text"].astype(str)       # Force all entries to be strings
# df

,text,label
0,ablaze wholesale market ablaze,1
1,ablaze always try bring heavy metal,0
2,ablaze africanbaze breaking newsnigeria flag s...,1
3,ablaze cry set ablaze,0
4,ablaze plus side look sky last night ablaze,0
...,...,...
7424,wrecked wrecked,0
7425,wrecked three day work theyve pretty wrecked h...,0
7426,wrecked fx forex trading cramer igers word wre...,0
7427,wrecked great atmosphere british lion gig toni...,0


In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
# dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 7429
})

In [ ]:
# print(dataset.column_names)

['text', 'label']


In [ ]:
# print(dataset[0])

{'text': 'ablaze wholesale market ablaze', 'label': 1}


In [ ]:
# print(type(dataset[0]["text"]))

<class 'str'>


In [ ]:
# df.dtypes

In [ ]:
# dataset = dataset.map(tokenize_function, batched=True)
# dataset = dataset.rename_column("label", "labels")
# dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/7429 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# dataset = dataset.map(tokenize_function, batched=True)
# dataset = dataset.rename_column("label", "labels")
# dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
# dataset

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7429
})

In [ ]:
# IMPORTANT: It may make sense to implement cross-validation here!
# IMPORANT: Make sure you commented out certain code!!!

In [ ]:
n = 5
kf = KFold(n_splits=n, shuffle=True, random_state=42)

acc_array_tr = [] # 'tr' is short for 'train'
f1_array_tr = []
precision_array_tr = []
recall_array_tr = []
roc_auc_tr = []

acc_array_val = []
f1_array_val = []
precision_array_val = []
recall_array_val = []
roc_auc_val = []
# roc_auc_array = [] # Maybe implement this later

for train_index, val_index in kf.split(dataset):
  train = dataset[train_index]
  val = dataset[val_index]

  train = Dataset.from_dict(train)
  val = Dataset.from_dict(val)

  print(train)
  print(val)

  # IMPORTANT: Is this the right place to put this?
  train = train.map(tokenize_function, batched=True)
  train = train.rename_column("label", "labels")
  train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

  # IMPORTANT: Is this the right place to put this?
  val = val.map(tokenize_function, batched=True)
  val = val.rename_column("label", "labels")
  val.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

  print(train)
  print(val)

  model = AutoModel.from_pretrained("vinai/bertweet-base")

  training_args = TrainingArguments (
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Add this!
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
  )

  trainer = Trainer (
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=tokenizer,
  )

  trainer.train()
  trainer.evaluate()

  # Predict on training data
  train_output = trainer.predict(train)
  train_preds = np.argmax(train_output.predictions, axis=1)  # Convert logits to predicted class

  # Predict on test data
  val_output = trainer.predict(val)
  val_preds = np.argmax(val_output.predictions, axis=1)

  # (Optional) True labels
  train_labels = train_output.label_ids
  val_labels = val_output.label_ids

  accuracy_tr = metrics.accuracy_score(train_labels, train_preds)
  f1_tr = metrics.f1_score(train_labels, train_preds)
  precision_tr = metrics.precision_score(train_labels, train_preds)
  recall_tr = metrics.recall_score(train_labels, train_preds)

  accuracy_val = metrics.accuracy_score(val_labels, val_preds)
  f1_val = metrics.f1_score(val_labels, val_preds)
  precision_val = metrics.precision_score(val_labels, val_preds)
  recall_val = metrics.recall_score(val_labels, val_preds)
  # roc_auc = metrics.roc_auc_score(val_labels, val_preds) # Probably not correct

  acc_array_tr.append(accuracy_tr)
  f1_array_tr.append(f1_tr)
  precision_array_tr.append(precision_tr)
  recall_array_tr.append(recall_tr)

  acc_array_val.append(accuracy_val)
  f1_array_val.append(f1_val)
  precision_array_val.append(precision_val)
  recall_array_val.append(recall_val)



Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 5943
})
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1486
})


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-54-59c89254e13c>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer (


Epoch,Training Loss,Validation Loss
1,0.479100,0.456562
2,0.371900,0.464361
3,0.298100,0.636020


Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 5943
})
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1486
})


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-54-59c89254e13c>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer (


Epoch,Training Loss,Validation Loss
1,0.475400,0.417790


In [ ]:
print("Training Accuracy Array: ")
print(acc_array_tr)
print("Training F1 Array: ")
print(f1_array_tr)
print("Training Precision Array: ")
print(precision_array_tr)
print("Training Recall Array: ")
print(recall_array_tr)

print("Training Accuracy mean: ")
print(np.mean(acc_array_tr))
print("Training F1 mean: ")
print(np.mean(f1_array_tr))
print("Training Precision mean: ")
print(np.mean(precision_array_tr))
print("Training Recall mean: ")
print(np.mean(recall_array_tr))

In [ ]:
print("Validation Accuracy Array: ")
print(acc_array_val)
print("Validation F1 Array: ")
print(f1_array_val)
print("Validation Precision Array: ")
print(precision_array_val)
print("Validation Recall Array: ")
print(recall_array_val)

print("Validation Accuracy mean: ")
print(np.mean(acc_array_val))
print("Validation F1 mean: ")
print(np.mean(f1_array_val))
print("Validation Precision mean: ")
print(np.mean(precision_array_val))
print("Validation Recall mean: ")
print(np.mean(recall_array_val))

In [ ]:
# dataset = dataset.train_test_split(test_size=0.2, seed=42)
# train_dataset = dataset["train"]
# eval_dataset = dataset["test"]

In [ ]:
# IMPORTANT: It may make sense to implement cross-validation here!

In [ ]:
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",  # Add this!
#     logging_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     save_total_limit=1,
#     load_best_model_at_end=True,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train,
#     eval_dataset=val,
#     tokenizer=tokenizer,
# )

In [ ]:
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# scores = cross_val_score(trainer, ...)

In [ ]:
# trainer.train()

In [ ]:
# trainer.evaluate()

In [ ]:
# Predict on new data
# text = "The plot was really boring."
# inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
# inputs = {k: v.to(model.device) for k, v in inputs.items()}
# outputs = model(**inputs)
# pred = torch.argmax(outputs.logits, dim=1).item()
# print("Prediction:", pred)

In [ ]:
# with torch.no_grad():
#     logits = model(**inputs).logits

In [ ]:
# predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]

In [ ]:
# train_dataset

In [ ]:
# test_dataset = eval_dataset

In [ ]:
# test_dataset

In [ ]:
# IMPORTANT: Don't forget to implement cross-validation!

# IMPORTANT: Is all of the data available in the prepended_v3_lemmatized.csv dataset?

In [ ]:
# import numpy as np

# # Predict on training data
# train_output = trainer.predict(train_dataset)
# train_preds = np.argmax(train_output.predictions, axis=1)  # Convert logits to predicted class

# # Predict on test data
# test_output = trainer.predict(test_dataset)
# test_preds = np.argmax(test_output.predictions, axis=1)

# # (Optional) True labels
# train_labels = train_output.label_ids
# test_labels = test_output.label_ids


In [ ]:
# test_labels

In [ ]:
# test_preds

In [ ]:
# metrics.accuracy_score(test_labels, test_preds)

In [ ]:
# metrics.f1_score(test_labels, test_preds)

In [ ]:
# IMPORTANT: Look into hyperparameter tuning! (Look at Dr. Roberts' slides and online?)

# IMPORTANT: Integrate this model into the main pipeline!

# IMPORANT: Commit your code!!!